# Sentiment Analysis with Deep Learning using BERT

### Project Outline

**Task 1**: Introduction

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

<img src="Images/BERT_diagrams.png" width="1000">

## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('Data/smile-annotations-final.csv', names = ['id', 'text', 'category'])
df.set_index('id', inplace = True)

In [3]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [4]:
df['text'].iloc[0]

'@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap'

### Looking into the distribution of the labels

In [5]:
df['category'].value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

1. We will only keep the tweets which have one label associated with them. 
2. We will also discard the tweets with "nocode" label, as they are ambiguous. 

In [6]:
# We can identify if a category has more than 2 emotions by searching for "|" in category string and exclude them.
df = df[~df['category'].str.contains('\|')]

In [7]:
df = df[df['category']!='nocode']

In [8]:
df['category'].value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

The distribution of classes is not balanced. Happy is dominating the class counts.

#### Assignment of a numerical id to each class

In [9]:
possible_labels = df['category'].unique()

In [10]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [11]:
df['label'] = df['category'].replace(label_dict)
df.head(10)

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1


## Task 3: Training/Validation Split

In [12]:
from sklearn.model_selection import train_test_split

**Our data is not balanced in terms of class counts. So we have to do a stratified split on "label"**

In [13]:
X_train, X_val, y_train, y_val = train_test_split(
                df.index.values,
                df.label.values,
                test_size = 0.15,
                random_state = 17,
                stratify = df.label.values
                )

### Set a flag to identify which rows belongs to training vs testing

In [14]:
df['data_type'] = ['not_set']*df.shape[0]

In [15]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [16]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

### Sanity Check

In [17]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## Task 4: Loading Tokenizer and Encoding our Data

In [18]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case=True)

In [20]:
encoded_data_train = tokenizer.batch_encode_plus(
                     df[df['data_type']=='train']['text'].values,
                     add_special_tokens = True,
                     return_attention_masks=True,
                     pad_to_max_length=True,
                     max_length=256, # max tweet length is of 256 words.
                     return_tensors='pt' # for pytorch
                        )

encoded_data_val = tokenizer.batch_encode_plus(
                     df[df['data_type']=='val']['text'].values,
                     add_special_tokens = True,
                     return_attention_masks=True,
                     pad_to_max_length=True,
                     max_length=256, # max tweet length is of 256 words.
                     return_tensors='pt' # for pytorch
                        )

input_ids_train = encoded_data_train['input_ids']
attention_masks_train =  encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df['data_type']=='train']['label'].values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val =  encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df['data_type']=='val']['label'].values)

### Setting up the training and validation dataset

In [21]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

### Sanity Check

In [22]:
len(dataset_train), len(dataset_val)

(1258, 223)

## Task 5: Setting up BERT Pretrained Model

In [23]:
from transformers import BertForSequenceClassification

In [24]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = len(label_dict),
                                                      output_attentions = False, output_hidden_states = False)

## Task 6: Creating Data Loaders

In [25]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

DataLoader == > provides a nice way to iterate through the dataset in batches.

RandomSampler, SequentialSampler ==> How to sample a dataset for a batch 
RandomSampler ==> for training, SequentialSampler ==> for validation

In [26]:
batch_size = 4
dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
dataloader_val = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=32)

## Task 7: Setting Up Optimizer and Scheduler

In [27]:
from transformers import AdamW, get_linear_schedule_with_warmup # AdamW ==> Adam with weight decay

In [28]:
optimizer = AdamW(
                model.parameters(),
                lr = 1e-5, #recommended by original Paper 2e-5 to 5e-5
                eps = 1e-8
                )

In [29]:
epochs = 10
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [30]:
import numpy as np

In [31]:
from sklearn.metrics import f1_score

**To use f1 score we would need to provide the predictions properly**

original preds will be a probability distribution like, [0.9, 0.05, 0.05, 0, 0, 0]

We need to convert the preds to binary vector like, [1, 0, 0, 0, 0, 0]

In [32]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [33]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k,v in label_dict.items()}
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        
        print (f'Class: {label_dict_inverse[label]}')
        print (f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [34]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [36]:
def evaluate(dataloader_val):

    model.eval() # Evaluation mode freezes all the weights of the models 
                 # temporarily and do not do backpropagation.
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [37]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0.
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave = False, disable = False)
    
    for batch in progress_bar:
        # Setting Model Gradients to 0
        model.zero_grad()
        # It is important if you are using GPU and Cuda
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids':       batch[0],
            'attention_mask':  batch[1],
            'labels':          batch[2]
        }
        
        outputs = model(**inputs) # It unpacks the dictionary straight into the inputs
        loss, logits = outputs
        loss_train_total += loss.item()
        loss.backward() # Backpropagate
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Gradient clipping
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))}) # See the avg. loss per batch
        
    torch.save(model.state_dict(), f'Models/Bert_ft_epoch{epoch}.model') #Saving the model at each epoch
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Validation F1 Score: {val_f1}')
    

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)



Epoch 1
Training loss: 0.7076102567452287
Validation loss: 0.6913102716207504
Validation F1 Score: 0.7475976937860345



Epoch 2
Training loss: 0.4420829193858755
Validation loss: 0.627915763429233
Validation F1 Score: 0.8230116011525148



Epoch 3
Training loss: 0.2831895442590827
Validation loss: 0.7023984289595059
Validation F1 Score: 0.8309142546263494



Epoch 4
Training loss: 0.188315298141069
Validation loss: 0.6909762322902679
Validation F1 Score: 0.8420533742001174



Epoch 5
Training loss: 0.11454379585542021
Validation loss: 0.7172981436763491
Validation F1 Score: 0.8561553280734875



Epoch 6
Training loss: 0.07089649212162291


KeyboardInterrupt: 

## Task 10: Loading and Evaluating our Model

**Using just trained till epoch5 model parameters**

In [38]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [42]:
model.load_state_dict(
            torch.load('Models/Bert_ft_epoch5.model',
                      map_location = torch.device('cpu'))
                    )

<All keys matched successfully>

In [44]:
val_loss, predictions, true_vals = evaluate(dataloader_val)


In [45]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 162/171

Class: not-relevant
Accuracy: 20/32

Class: angry
Accuracy: 8/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 1/5

Class: surprise
Accuracy: 2/5



**Using Previously GPU trained model parameters**

In [46]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [47]:
model.load_state_dict(
            torch.load('Models/finetuned_bert_epoch_1_gpu_trained.model',
                      map_location = torch.device('cpu'))
                    )

<All keys matched successfully>

In [48]:
val_loss, predictions, true_vals = evaluate(dataloader_val)

In [49]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 163/171

Class: not-relevant
Accuracy: 20/32

Class: angry
Accuracy: 7/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 4/5

Class: surprise
Accuracy: 2/5

